In [5]:
# Import stuff
import torch as t
import numpy as np
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import plotly.express as px
import einops
import plotly.graph_objects as go 
from functools import partial
import tqdm.auto as tqdm
import circuitsvis as cv
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, ActivationCache
from transformer_lens.components import Embed, Unembed, LayerNorm, MLP
from fancy_einsum import einsum
from jaxtyping import Float, Int, Bool
import re
import random


In [2]:

def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [3]:
## turn off AD to save memory, since we're focusing on model inference here 
t.set_grad_enabled(False)

device = 'cuda' if t.cuda.is_available() else 'cpu'
model = HookedTransformer.from_pretrained('gpt2-small', device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


Studying the head outputs on different distributions

Create the datasets 

In [87]:
proper_nouns = [
    " Goose", " Church",
    " Google", " Chair",
    " Bag", " Statue",
    " Lamp", " Flower"
]

nouns = [
    " goose", " church",
    " google", " chair",
    " bag", " statue",
    " lamp", " flower"
]

multi_names_religious = [
    " Mary", " Joseph",
    " Abraham", " Paul",
    " Isaac", " Noah",
    " Jacob", " Jesus"
]

multi_names_places = [
    " Paris", " London",
    " Madison", " Phoenix",
    " Devon", " Florence",
    " Austin", " Brooklyn"
]

test = proper_nouns
[model.to_single_token(test[i]) for i in range(len(test))]


[46317, 4564, 3012, 9369, 20127, 43330, 28607, 20025]

In [233]:



def get_dataset(N, names):
    prompts = []
    # List of answers, in the format (correct, incorrect)
    answers = []
    # List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
    answer_tokens = []
    for _ in range(N):
        S1, S2 = random.sample(names,2)
        answers.append((S1,S2))
        answers.append((S2,S1))
        prompt1 = f'When {S1} and {S2} went to the shops,{S2} gave the bag to'
        prompt2 = f'When {S1} and {S2} went to the shops,{S1} gave the bag to'
        prompts.append(prompt1)
        prompts.append(prompt2)

        answer_tok_1 = model.to_single_token(S1)
        answer_tok_2 = model.to_single_token(S2)

        answer_tokens.append((answer_tok_1, answer_tok_2))
        answer_tokens.append((answer_tok_2, answer_tok_1))
    assert len(set([len(model.to_str_tokens(prompt)) for prompt in prompts])) == 1
    answer_tokens = t.tensor(answer_tokens).to(device)
    return prompts, answers, answer_tokens



In [234]:
prompts_rel, ans_rel, ans_toks_rel = get_dataset(10,multi_names_religious)
prompts_pl, ans_pl, ans_toks_pl = get_dataset(10,multi_names_places)
prompts_n, ans_n, ans_toks_n = get_dataset(10,nouns)
prompts_pn, ans_pn, ans_toks_pn = get_dataset(10,proper_nouns)



In [235]:
from rich.table import Table, Column
from rich import print as rprint


In [236]:
control_prompts = ['When John and Mary went to the shops, John gave the bag to',
 'When John and Mary went to the shops, Mary gave the bag to',
 'When Tom and James went to the park, James gave the ball to',
 'When Tom and James went to the park, Tom gave the ball to',
 'When Dan and Sid went to the shops, Sid gave an apple to',
 'When Dan and Sid went to the shops, Dan gave an apple to',
 'After Martin and Amy went to the park, Amy gave a drink to',
 'After Martin and Amy went to the park, Martin gave a drink to']

control_answers = [(' Mary', ' John'),
 (' John', ' Mary'),
 (' Tom', ' James'),
 (' James', ' Tom'),
 (' Dan', ' Sid'),
 (' Sid', ' Dan'),
 (' Martin', ' Amy'),
 (' Amy', ' Martin')]

control_ans_prompts = t.tensor([[ 5335,  1757],
        [ 1757,  5335],
        [ 4186,  3700],
        [ 3700,  4186],
        [ 6035, 15686],
        [15686,  6035],
        [ 5780, 14235],
        [14235,  5780]])



In [237]:
print(model(prompts_n).shape)

def ave_logit_diff(prompts, answer_tokens, per_prompt = False):
    final_logits = model(prompts)[:,-1,:]
    answer_logits = final_logits.gather(dim = -1, index = answer_tokens)
    print(final_logits.shape)
    answer_logit_diff = answer_logits[:,0] - answer_logits[:,1]
    if per_prompt:
        return answer_logit_diff 
    else:
        return answer_logit_diff.mean()

ave_logit_diff(prompts_n, ans_toks_n)

torch.Size([20, 17, 50257])
torch.Size([20, 50257])


tensor(0.4340)

In [238]:
def make_table(prompts, answers, answer_tokens, title):
    cols = [
        "Prompt", 
        Column("Correct", style="rgb(0,200,0) bold"), 
        Column("Incorrect", style="rgb(255,0,0) bold"), 
        Column("Logit Difference", style="bold")
    ]
    logit_diffs = ave_logit_diff(prompts,answer_tokens, per_prompt = True)
    
    ave_logits = ave_logit_diff(prompts,answer_tokens, per_prompt = False)
    logit_diff_table = Table(*cols, title=title + f": Ave logit diff = {ave_logits.item():.3f}")
    logit_diffs = ave_logit_diff(prompts,answer_tokens, per_prompt = True)
    
    ave_logits = ave_logit_diff(prompts,answer_tokens, per_prompt = False)
                                 
    ave_logit_diff(prompts, answer_tokens)
    for prompt, ans, logit_diff in zip(prompts, answers,logit_diffs):
        logit_diff_table.add_row(prompt, ans[0], ans[1], f"{logit_diff.item():.3f}")
    rprint(logit_diff_table)

In [239]:
make_table(control_prompts, control_answers, control_ans_prompts, "Control Names")

torch.Size([8, 50257])
torch.Size([8, 50257])
torch.Size([8, 50257])
torch.Size([8, 50257])
torch.Size([8, 50257])


                                  Control Names: Ave logit diff = 3.552                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                        ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When John and Mary went to the shops, John gave the bag to    │  Mary   │  John     │ 3.337            │
│ When John and Mary went to the shops, Mary gave the bag to    │  John   │  Mary     │ 3.202            │
│ When Tom and James went to the park, James gave the ball to   │  Tom    │  James    │ 2.709            │
│ When Tom and James went to the park, Tom gave the ball to     │  James  │  Tom      │ 3.797            │
│ When Dan and Sid went to the shops, Sid gave an apple to      │  Dan    │  Sid      │ 1.720            │
│ When Dan and Sid went to the shops, Dan gave an apple to      │  Sid    │  Dan      │ 5.281            │
│ After Martin and Amy went to the park, Amy gave a drink to    │  Martin │  Amy      │ 2.601            │
│ After Martin and Amy went to the park, Martin gave a drink to │  Amy    │  Martin   │ 5.767            │
└───────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

In [245]:
#[(model.to_str_tokens(ans_toks_n[k],ans_n[k]), model.to_str_tokens(ans_toks_n[k]),model.to_str_tokens(ans_n[k]), ans_n[k]) for k in range(len(ans_toks_n))]

test = ans_toks_n[10]
test_ans = ans_n[10]
[model.tokenizer.decode(test[i]) for i in range(len(test))], model.to_str_tokens(test), ans_n[10]


([' statue', ' lamp'], [' statue', ' lamp'], (' statue', ' lamp'))

In [241]:
make_table(prompts_n, ans_n, ans_toks_n, "nouns")
make_table(prompts_pn, ans_pn, ans_toks_pn, "proper nouns")
make_table(prompts_rel, ans_rel, ans_toks_rel, "Religious Names")
make_table(prompts_pl, ans_pl, ans_toks_pl, "Place Names")

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                         nouns: Ave logit diff = 0.434                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                             ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  goose and  lamp went to the shops, lamp gave the bag to      │  goose  │  lamp     │ -1.035           │
│ When  goose and  lamp went to the shops, goose gave the bag to     │  lamp   │  goose    │ 4.286            │
│ When  google and  statue went to the shops, statue gave the bag to │  google │  statue   │ -2.737           │
│ When  google and  statue went to the shops, google gave the bag to │  statue │  google   │ 0.106            │
│ When  chair and  flower went to the shops, flower gave the bag to  │  chair  │  flower   │ -0.595           │
│ When  chair and  flower went to the shops, chair gave the bag to   │  flower │  chair    │ 3.918            │
│ When  chair and  lamp went to the shops, lamp gave the bag to      │  chair  │  lamp     │ -0.861           │
│ When  chair and  lamp went to the shops, chair gave the bag to     │  lamp   │  chair    │ 3.106            │
│ When  church and  chair went to the shops, chair gave the bag to   │  church │  chair    │ 2.318            │
│ When  church and  chair went to the shops, church gave the bag to  │  chair  │  church   │ -2.298           │
│ When  statue and  lamp went to the shops, lamp gave the bag to     │  statue │  lamp     │ -1.542           │
│ When  statue and  lamp went to the shops, statue gave the bag to   │  lamp   │  statue   │ 3.675            │
│ When  church and  flower went to the shops, flower gave the bag to │  church │  flower   │ 1.274            │
│ When  church and  flower went to the shops, church gave the bag to │  flower │  church   │ -0.807           │
│ When  statue and  chair went to the shops, chair gave the bag to   │  statue │  chair    │ 0.999            │
│ When  statue and  chair went to the shops, statue gave the bag to  │  chair  │  statue   │ 0.657            │
│ When  lamp and  google went to the shops, google gave the bag to   │  lamp   │  google   │ -2.178           │
│ When  lamp and  google went to the shops, lamp gave the bag to     │  google │  lamp     │ 0.849            │
│ When  church and  bag went to the shops, bag gave the bag to       │  church │  bag      │ 0.261            │
│ When  church and  bag went to the shops, church gave the bag to    │  bag    │  church   │ -0.718           │
└────────────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                     proper nouns: Ave logit diff = 2.838                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                             ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  Lamp and  Statue went to the shops, Statue gave the bag to   │  Lamp   │  Statue   │ 4.809            │
│ When  Lamp and  Statue went to the shops, Lamp gave the bag to     │  Statue │  Lamp     │ 1.914            │
│ When  Bag and  Chair went to the shops, Chair gave the bag to      │  Bag    │  Chair    │ 5.628            │
│ When  Bag and  Chair went to the shops, Bag gave the bag to        │  Chair  │  Bag      │ -1.143           │
│ When  Goose and  Flower went to the shops, Flower gave the bag to  │  Goose  │  Flower   │ 5.803            │
│ When  Goose and  Flower went to the shops, Goose gave the bag to   │  Flower │  Goose    │ 5.488            │
│ When  Goose and  Flower went to the shops, Flower gave the bag to  │  Goose  │  Flower   │ 5.803            │
│ When  Goose and  Flower went to the shops, Goose gave the bag to   │  Flower │  Goose    │ 5.488            │
│ When  Statue and  Google went to the shops, Google gave the bag to │  Statue │  Google   │ -3.054           │
│ When  Statue and  Google went to the shops, Statue gave the bag to │  Google │  Statue   │ 3.408            │
│ When  Google and  Lamp went to the shops, Lamp gave the bag to     │  Google │  Lamp     │ 0.578            │
│ When  Google and  Lamp went to the shops, Google gave the bag to   │  Lamp   │  Google   │ 0.967            │
│ When  Chair and  Flower went to the shops, Flower gave the bag to  │  Chair  │  Flower   │ 2.027            │
│ When  Chair and  Flower went to the shops, Chair gave the bag to   │  Flower │  Chair    │ 5.775            │
│ When  Bag and  Chair went to the shops, Chair gave the bag to      │  Bag    │  Chair    │ 5.628            │
│ When  Bag and  Chair went to the shops, Bag gave the bag to        │  Chair  │  Bag      │ -1.143           │
│ When  Flower and  Statue went to the shops, Statue gave the bag to │  Flower │  Statue   │ 3.393            │
│ When  Flower and  Statue went to the shops, Flower gave the bag to │  Statue │  Flower   │ 2.502            │
│ When  Statue and  Chair went to the shops, Chair gave the bag to   │  Statue │  Chair    │ 1.331            │
│ When  Statue and  Chair went to the shops, Statue gave the bag to  │  Chair  │  Statue   │ 1.564            │
└────────────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                     Religious Names: Ave logit diff = 3.421                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                              ┃ Correct  ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  Jacob and  Paul went to the shops, Paul gave the bag to       │  Jacob   │  Paul     │ 4.659            │
│ When  Jacob and  Paul went to the shops, Jacob gave the bag to      │  Paul    │  Jacob    │ 4.560            │
│ When  Noah and  Jesus went to the shops, Jesus gave the bag to      │  Noah    │  Jesus    │ 4.621            │
│ When  Noah and  Jesus went to the shops, Noah gave the bag to       │  Jesus   │  Noah     │ 2.645            │
│ When  Paul and  Jesus went to the shops, Jesus gave the bag to      │  Paul    │  Jesus    │ 3.775            │
│ When  Paul and  Jesus went to the shops, Paul gave the bag to       │  Jesus   │  Paul     │ 0.811            │
│ When  Isaac and  Abraham went to the shops, Abraham gave the bag to │  Isaac   │  Abraham  │ 3.095            │
│ When  Isaac and  Abraham went to the shops, Isaac gave the bag to   │  Abraham │  Isaac    │ 1.792            │
│ When  Paul and  Joseph went to the shops, Joseph gave the bag to    │  Paul    │  Joseph   │ 2.202            │
│ When  Paul and  Joseph went to the shops, Paul gave the bag to      │  Joseph  │  Paul     │ 4.792            │
│ When  Paul and  Noah went to the shops, Noah gave the bag to        │  Paul    │  Noah     │ 3.363            │
│ When  Paul and  Noah went to the shops, Paul gave the bag to        │  Noah    │  Paul     │ 3.928            │
│ When  Isaac and  Jesus went to the shops, Jesus gave the bag to     │  Isaac   │  Jesus    │ 4.036            │
│ When  Isaac and  Jesus went to the shops, Isaac gave the bag to     │  Jesus   │  Isaac    │ 3.647            │
│ When  Paul and  Jesus went to the shops, Jesus gave the bag to      │  Paul    │  Jesus    │ 3.775            │
│ When  Paul and  Jesus went to the shops, Paul gave the bag to       │  Jesus   │  Paul     │ 0.811            │
│ When  Isaac and  Paul went to the shops, Paul gave the bag to       │  Isaac   │  Paul     │ 3.371            │
│ When  Isaac and  Paul went to the shops, Isaac gave the bag to      │  Paul    │  Isaac    │ 5.887            │
│ When  Joseph and  Mary went to the shops, Mary gave the bag to      │  Joseph  │  Mary     │ 5.585            │
│ When  Joseph and  Mary went to the shops, Joseph gave the bag to    │  Mary    │  Joseph   │ 1.064            │
└─────────────────────────────────────────────────────────────────────┴──────────┴───────────┴──────────────────┘

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                        Place Names: Ave logit diff = 3.132                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                               ┃ Correct   ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  Devon and  Florence went to the shops, Florence gave the bag   │  Devon    │  Florence │ 3.998            │
│ to                                                                   │           │           │                  │
│ When  Devon and  Florence went to the shops, Devon gave the bag to   │  Florence │  Devon    │ 7.686            │
│ When  Florence and  Paris went to the shops, Paris gave the bag to   │  Florence │  Paris    │ 2.874            │
│ When  Florence and  Paris went to the shops, Florence gave the bag   │  Paris    │  Florence │ 2.317            │
│ to                                                                   │           │           │                  │
│ When  Brooklyn and  Austin went to the shops, Austin gave the bag to │  Brooklyn │  Austin   │ -0.968           │
│ When  Brooklyn and  Austin went to the shops, Brooklyn gave the bag  │  Austin   │  Brooklyn │ 6.529            │
│ to                                                                   │           │           │                  │
│ When  Austin and  Phoenix went to the shops, Phoenix gave the bag to │  Austin   │  Phoenix  │ 3.024            │
│ When  Austin and  Phoenix went to the shops, Austin gave the bag to  │  Phoenix  │  Austin   │ 3.813            │
│ When  London and  Brooklyn went to the shops, Brooklyn gave the bag  │  London   │  Brooklyn │ -3.078           │
│ to                                                                   │           │           │                  │
│ When  London and  Brooklyn went to the shops, London gave the bag to │  Brooklyn │  London   │ 3.658            │
│ When  Brooklyn and  Austin went to the shops, Austin gave the bag to │  Brooklyn │  Austin   │ -0.968           │
│ When  Brooklyn and  Austin went to the shops, Brooklyn gave the bag  │  Austin   │  Brooklyn │ 6.529            │
│ to                                                                   │           │           │                  │
│ When  Austin and  Madison went to the shops, Madison gave the bag to │  Austin   │  Madison  │ 2.843            │
│ When  Austin and  Madison went to the shops, Austin gave the bag to  │  Madison  │  Austin   │ 5.580            │
│ When  Phoenix and  Paris went to the shops, Paris gave the bag to    │  Phoenix  │  Paris    │ 1.135            │
│ When  Phoenix and  Paris went to the shops, Phoenix gave the bag to  │  Paris    │  Phoenix  │ 5.022            │
│ When  Brooklyn and  Florence went to the shops, Florence gave the    │  Brooklyn │  Florence │ -1.866           │
│ bag to                                                               │           │           │                  │
│ When  Brooklyn and  Florence went to the shops, Brooklyn gave the    │  Florence │  Brooklyn │ 7.412            │
│ bag to                                                               │           │           │                  │
│ When  Phoenix and  Madison went to the shops, Madison gave the bag   │  Phoenix  │  Madison  │ 1.461            │
│ to                                                                   │           │           │                  │
│ When  Phoenix and  Madison went to the shops, Phoenix gave the bag   │  Madison  │  Phoenix  │ 5.644            │
│ to                                                                   │           │           │                  │
└──────────────────────────────────────────────────────────────────────┴───────────┴───────────┴──────────────────┘

Looking at the top-k logits for each prompt

In [268]:
prompts = prompts_pn
k = 3

for sample_prompt in prompts:
    sample_logits = model(sample_prompt)
    sample_probs = t.softmax(sample_logits[0, -1], dim = -1)
    vals, ids = t.topk(sample_probs,k)

    print(f"Prompt = {sample_prompt}")
    for i in range(k):
            print(f"Top {i}th logit. prob = {vals[i]:.2%}, token = {model.tokenizer.decode(ids[i])}")



Prompt = When  Lamp and  Statue went to the shops, Statue gave the bag to
Top 0th logit. prob = 33.93%, token =  Lamp
Top 1th logit. prob = 15.14%, token =  
Top 2th logit. prob = 8.23%, token =  the
Prompt = When  Lamp and  Statue went to the shops, Lamp gave the bag to
Top 0th logit. prob = 24.99%, token =  
Top 1th logit. prob = 20.68%, token =  the
Top 2th logit. prob = 3.99%, token =  Statue
Prompt = When  Bag and  Chair went to the shops, Chair gave the bag to
Top 0th logit. prob = 21.65%, token =  Bag
Top 1th logit. prob = 18.90%, token =  
Top 2th logit. prob = 7.51%, token =  the
Prompt = When  Bag and  Chair went to the shops, Bag gave the bag to
Top 0th logit. prob = 18.82%, token =  
Top 1th logit. prob = 16.44%, token =  the
Top 2th logit. prob = 4.62%, token =  a
Prompt = When  Goose and  Flower went to the shops, Flower gave the bag to
Top 0th logit. prob = 32.10%, token =  
Top 1th logit. prob = 17.60%, token =  Goose
Top 2th logit. prob = 8.15%, token =  the
Prompt = W